In [1]:
import re
from typing import List
from datetime import datetime
import pandas as pd

def chunk_and_merge_h1(md_text: str) -> List[str]:
    """
    1단계: 모든 헤더(#~####)에서 청킹
    2단계: H1이 있는 청크는 다음 청크와 합치기
    """
    # 1단계: 모든 헤더 레벨에서 청킹
    header_pattern = re.compile(r'^(#{1,4})\s+(.+)$', re.MULTILINE)
    matches = list(header_pattern.finditer(md_text))

    if not matches:
        return [md_text.strip()]

    # 초기 청킹
    chunks = []

    # 첫 번째 헤더 이전 내용
    first_start = matches[0].start()
    if first_start > 0:
        intro = md_text[:first_start].strip()
        if intro:
            chunks.append(intro)

    # 각 헤더 구간으로 청킹
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(md_text)
        chunk_content = md_text[start:end].strip()
        chunks.append(chunk_content)

    # 2단계: H1이 있는 청크를 다음 청크와 합치기
    merged_chunks = []
    i = 0

    while i < len(chunks):
        current_chunk = chunks[i]

        # 현재 청크가 H1으로 시작하는지 확인
        if re.match(r'^#\s+', current_chunk):
            # H1이면 다음 청크와 합치기
            if i + 1 < len(chunks):
                merged_chunk = current_chunk + "\n\n" + chunks[i + 1]
                merged_chunks.append(merged_chunk)
                i += 2  # 두 청크를 처리했으므로 2 증가
            else:
                # 마지막 청크가 H1이면 그대로 추가
                merged_chunks.append(current_chunk)
                i += 1
        else:
            # H1이 아니면 그대로 추가
            merged_chunks.append(current_chunk)
            i += 1

    return merged_chunks

In [13]:
def load_markdown_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [14]:
def load_and_chunk_markdown(file_path: str) -> List[str]:
    md_text = load_markdown_file(file_path)
    return chunk_and_merge_h1(md_text)

# CS 청킹

In [15]:
cs_path = "/Users/kimbwook/PycharmProjects/KHU-EE-RAG/data/parsed/cs_curriculum/cs_curriculum.md"

In [16]:
cs_chunks = load_and_chunk_markdown(cs_path)

In [17]:
len(cs_chunks)

72

### Chunks to Corpus

In [18]:
cs_doc_ids = [f"cs_chunk_{i+1}" for i in range(len(cs_chunks))]

In [19]:
cs_paths = [cs_path for _ in cs_chunks]

In [20]:
cs_metadata = {
    "last_modified_datetime": datetime.now(),
}
cs_metadatas = [cs_metadata for _ in cs_chunks]

In [22]:
cs_df = pd.DataFrame({
    "doc_id": cs_doc_ids,
    "contents": cs_chunks,
    "path": cs_paths,
    "metadata": cs_metadatas
})

In [23]:
cs_df.to_parquet("../../data/corpus/cs.parquet", index=False)

# EE 청킹

In [24]:
ee_path = "/Users/kimbwook/PycharmProjects/KHU-EE-RAG/data/parsed/ee_curriculum/ee_curriculum.md"

In [25]:
ee_chunks = load_and_chunk_markdown(ee_path)

In [26]:
len(ee_chunks)

71

In [27]:
ee_chunks[0]

'# 전자정보대학 교육과정\n\n#### 대학소개\n\n◩ 21세기 네오르네상스 시대의 4차 산업혁명을 주도해 나갈 공학기술 인재 양성을 지향하는 전자정보대학은 1999년에 신설되 었으며, 전자공학과와 생체의공학과로 이루어져 있다. 차세대 산업혁명은 정보통신, 바이오, 나노기술 등의 다양한 첨단 기술 의 융합을 통해 초연결, 초지능, 초융합의 고도 기술사회의 도래를 예고하고 있다. 따라서 전자정보대학에서는 이와 같은 미래 기술 사회를 선도하고, 미래 가치를 창출하며, 차세대 산업혁명을 주도할 세계적 수준의 공학 인재를 양성하기 위하여 교육과 연구에 정진하고 있다. 전자정보대학에서는 학과별로 특성화된 교육 및 연구가 이루어지고 있으며, 또한 다양한 타 전공과의연계를 통한 학제간 융합 교육 및 연구가 진행되고 있다.'

### Chunks to Corpus

In [28]:
ee_doc_ids = [f"ee_chunk_{i+1}" for i in range(len(ee_chunks))]

In [29]:
ee_paths = [ee_path for _ in ee_chunks]

In [30]:
ee_metadata = {
    "last_modified_datetime": datetime.now(),
}
ee_metadatas = [ee_metadata for _ in ee_chunks]

In [31]:
ee_df = pd.DataFrame({
    "doc_id": ee_doc_ids,
    "contents": ee_chunks,
    "path": ee_paths,
    "metadata": ee_metadatas
})

In [32]:
ee_df.to_parquet("../../data/corpus/ee.parquet", index=False)

# Concatenate CS and EE

In [33]:
pd.concat([cs_df, ee_df]).to_parquet("../../data/corpus/corpus.parquet", index=False)

In [37]:
df = pd.read_parquet("../../data/corpus/corpus.parquet")
df

,doc_id,contents,path,metadata
0,cs_chunk_1,# 소프트웨어융합대학 교육과정\n\n대학소개\n\n- ◩ 소프트웨어융합대학은4차산업...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:30...
1,cs_chunk_2,#### **2. 교육목표**\n\n- 1)소프트웨어중심기업/학계및소프트웨어관련창업...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:30...
2,cs_chunk_3,#### **3. 설치학과**\n\n- 가. 컴퓨터공학과\n- 나. 소프트웨어융합학과,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:30...
3,cs_chunk_4,#### **4. 대학 졸업 요건**\n<!DOCTYPE html>\n<html l...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:30...
4,cs_chunk_5,# **5. 전공별 교육과정 편성 교과목수**\n\n| 학과/프로그램명 | ...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:30...
...,...,...,...,...
138,ee_chunk_67,#### **종합설계(전자공학)(Capstone Design)**\n\n이 과목에...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:36...
139,ee_chunk_68,#### **실전문제연구종합설계 (X-CorpsCapstone Design)**\...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:36...
140,ee_chunk_69,#### **졸업논문(전자공학)(Graduation Thesis)**\n\n이 과...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:36...
141,ee_chunk_70,# **대학원 과목 수강 신청 절차 및 수강 가능 교과목**\n\n#### **대...,/Users/kimbwook/PycharmProjects/KHU-EE-RAG/dat...,{'last_modified_datetime': 2025-07-25 16:47:36...
